In [1]:
import torch
torch.cuda.is_available()

True

In [23]:
from datasets import load_dataset

dataset = load_dataset("uit-nlp/vietnamese_students_feedback")

Found cached dataset vietnamese_students_feedback (/home/shay/.cache/huggingface/datasets/uit-nlp___vietnamese_students_feedback/default/1.0.0/b01aa0fea00f1179ef9c093babb5e91f44bc60f19ded600754a08e018503e40f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Fsoft-AIC/videberta-xsmall")

In [26]:
tokenizer

DebertaV2TokenizerFast(name_or_path='Fsoft-AIC/videberta-xsmall', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [28]:
def preprocess_function(row):
    return tokenizer(row["sentence"], truncation=True)

dataset = dataset.remove_columns("topic")

tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['validation'].map(preprocess_function, batched=True)

tokenized_train = tokenized_train.rename_columns({'sentiment': 'labels'})
tokenized_train = tokenized_train.rename_columns({'sentence': 'text'})

tokenized_test = tokenized_test.rename_columns({'sentiment': 'labels'})
tokenized_test = tokenized_test.rename_columns({'sentence': 'text'})

Map:   0%|          | 0/11426 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1583 [00:00<?, ? examples/s]

In [29]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [30]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("Fsoft-AIC/videberta-xsmall", num_labels=2)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at Fsoft-AIC/videberta-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [32]:
from huggingface_hub import notebook_login
notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
from transformers import TrainingArguments, Trainer
 
repo_name = "videberta-sentiment-analysis"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=64,
   per_device_eval_batch_size=64,
   num_train_epochs=300,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [30]:
!conda list | grep transformers[torch] 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
!conda list | grep accelerate 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
accelerate                0.21.0                   pypi_0    pypi


In [32]:
trainer.evaluate()

NameError: name 'trainer' is not defined

In [ ]:
trainer.push_to_hub()